In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw2'))
	print(os.getcwd())
except:
	pass

/home/renjiezhu/school_workspace/recommender_system/hw2


# Homework 2 - Renjie Zhu - A53266114

In [2]:
import scipy as sp
import numpy as np
from sklearn import linear_model
from scipy.io import arff
from io import StringIO

In [3]:
# raw, meta = arff.loadarff('5year.arff')
# X = raw["class"].astype(int)
# A = np.array([(raw["Attr"+str(i+1)]) for i in range(64)]).T
# A = np.nan_to_num(A)

In [4]:
f = open("5year.arff", 'r')

# Reading in data
while not '@data' in f.readline():
    pass

dataset = []
for l in f:
    if '?' in l: # Missing entry
        continue
    l = l.split(',')
    values = [1] + [float(x) for x in l]
    values[-1] = values[-1] > 0 # Convert to bool
    dataset.append(values)

A = np.array([d[:-1] for d in dataset])
X = np.array([d[-1] for d in dataset])

In [5]:
model = linear_model.LogisticRegression(C=1.0, solver='lbfgs',max_iter=10000)
model.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
pred = model.predict(A)
print(f"Accuracy: {sum(pred==X)/len(X)*100}%.")
# sum(pred==X)

Accuracy: 96.83272847245134%.


In [7]:
def BER(pred, X):
    TP = sum(np.logical_and(pred, X))
    FP = sum(np.logical_and(pred, np.logical_not(X)))
    TN = sum(np.logical_and(np.logical_not(pred), np.logical_not(X)))
    FN = sum(np.logical_and(np.logical_not(pred), X))
    return (0.5 * (FN / (FN + TN) + FP / (FP + TP)))

print(f"BER: {BER(pred, X)*100}%.")


BER: 15.810407689758039%.


In [8]:
AX = np.concatenate((A,X.reshape(-1,1)),axis=1)
np.random.shuffle(AX)

In [9]:
v = len(AX) // 2
t = len(AX) // 4

test_set = AX[:t]
valid_set = AX[t+1:v]
train_set = AX[v+1:]

A = train_set[:,:-1]
X = train_set[:,-1]


In [10]:
model_balanced = linear_model.LogisticRegression(C=1,class_weight="balanced",solver='lbfgs', max_iter=10000)
model_balanced.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
p_train = model_balanced.predict(A)
p_test = model_balanced.predict(test_set[:,:-1])
p_valid = model_balanced.predict(valid_set[:,:-1])

In [12]:
print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

Training set:	Accuracy: 84.0924092409241%	BER: 42.77368897240491%
Test set:	Accuracy: 81.63804491413474%	BER: 46.13916789585821%
Validation set:	Accuracy: 82.1664464993395%	BER: 44.18356997971602%


In [13]:
# model.fit(A, X)
# p_train = model.predict(A)
# p_test = model.predict(test_set[:,:-1])
# p_valid = model.predict(valid_set[:,:-1])
# print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
# print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
# print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

In [14]:
C = [1E-4, 1E-3, 1E-2, 1E-1, 1, 1E1, 1E2, 1E3, 1E4]
models = [linear_model.LogisticRegression(C=i, class_weight="balanced", solver='lbfgs', max_iter=10000) for i in C]

In [15]:
for c, aModel in zip(C,models):
    aModel.fit(A, X)
    p_train = aModel.predict(A)
    p_test = aModel.predict(test_set[:,:-1])
    p_valid = aModel.predict(valid_set[:,:-1])
    print(f"C = {c}")
    print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
    print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
    print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

C = 0.0001
Training set:	Accuracy: 83.36633663366337%	BER: 44.42129316480117%
Test set:	Accuracy: 82.82694848084545%	BER: 45.02662907268171%
Validation set:	Accuracy: 78.86393659180978%	BER: 47.13816276071935%
C = 0.001
Training set:	Accuracy: 85.41254125412541%	BER: 43.08704010354213%
Test set:	Accuracy: 84.5442536327609%	BER: 44.73952945545211%
Validation set:	Accuracy: 82.82694848084545%	BER: 44.99293260822692%
C = 0.01
Training set:	Accuracy: 84.48844884488449%	BER: 43.124279746470755%
Test set:	Accuracy: 83.09114927344781%	BER: 45.30069619315657%
Validation set:	Accuracy: 83.09114927344781%	BER: 43.842921027370004%
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C = 0.1
Training set:	Accuracy: 84.55445544554455%	BER: 42.58391107851025%
Test set:	Accuracy: 82.03434610303832%	BER: 46.394833236938496%
Validation set:	Accurac

In [ ]:
weights = [1.0] * len(ytrain)
mod = linear_model.LogisticRegression(C=1, solver='lbfgs')
mod.fit(Xtrain, ytrain, sample_weight=weights)